# Areal Interpolation

<a target="_blank" href="https://colab.research.google.com/github/knaaptime/uppp135-winter26-assn/blob/main/week4/interpolation.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [ ]:
import geopandas as gpd
import pandas as pd
from fsspec import filesystem
from tobler.area_weighted import area_interpolate
from tobler.dasymetric import masked_area_interpolate
from tobler.dasymetric import extract_raster_features
from tobler.util  import h3fy

## Census Tracts and Cities in OC

In [ ]:
fs = filesystem("https")
tracts = gpd.read_parquet("https://github.com/oturns/example_datasets/raw/refs/heads/main/acs/ca_tracts_2021.pq", filesystem=fs)
oc_tracts = tracts[tracts.geoid.str.startswith('06059')]

# this is a shapefile from the OC open data portal <https://data-ocpw.opendata.arcgis.com/datasets/60119fce76d74dc08c3aa455f34b2b4d_0/explore?
cities = gpd.read_file("https://github.com/knaaptime/uppp135-winter26-assn/raw/refs/heads/main/week4/OCTraffic_Cities.zip")
# dont worry about this part; there's a small error in the data and this is how we fix it
cities = cities.set_geometry(cities.buffer(0))

In [ ]:
oc_tracts.head()

In [ ]:
cities.head()

Since we got these datasets from different places, it's a good idea to check their CRS

In [ ]:
oc_tracts.crs

In [ ]:
cities.crs

uh oh. These two do not share a single CRS... `oc_tracts` is EPSG 4269 and `cities` is EPSG 3857

so we need to convert one (or both).  Which one should we use? 

One option that works well, especially in the continental U.S., is the [Universal Transverse Mercator (UTM)](https://en.wikipedia.org/wiki/Universal_Transverse_Mercator_coordinate_system) zonal system; we just need to know which UTM zone our data falls inside, then we can convert all our geodataframes into that system. Geopandas gives us a convenient way to do this

![UTM Zones in the continental U.S.](https://upload.wikimedia.org/wikipedia/commons/thumb/8/8d/Utm-zones-USA.svg/2560px-Utm-zones-USA.svg.png)

In [ ]:
socal_crs = cities.estimate_utm_crs()

In [ ]:
socal_crs

Geopandas says that our data falls inside UTM Zone 11N (which looks right in the image above). That's also known as EPSG 32611.

Now, we convert `cities` and `oc_tracts` into that system.

In [ ]:
cities = cities.to_crs(socal_crs)
oc_tracts = oc_tracts.to_crs(socal_crs)

In [ ]:
m = oc_tracts.explore(tiles='cartodb positron')
cities.explore('city', categorical=True, alpha=0.4,  m=m )

## Area-Weighted Interpolation

In the maps above we can see that the census tract boundaries do not nest perfectly inside the city boundaries. Look, for example, where Orange, Santa Ana, Garden Grove, and Anaheim all meet.

Now we have all this rich information at the census tract level and we want to estimate it somehow for each city. Area-weighted interpolation is one way we can do that. First lets remember our task.

we have data at the tract level in `oc_tracts` (our source) that we want to transfer into the city-level (our target), `cities`. Imagine we're interested in *total population* in our tracts data as `n_total_pop` and educational attainment, stored as a column in the tracts dataframe called `p_edu_college_greater`

In [ ]:
area_interpolate?

remember that we need to treat extensive and intensive variables separately

In [ ]:
extensive_variables = ['n_total_pop']
intensive_variables = ['p_edu_college_greater']

One of the columns in `cities` is called "city". Like you'd expect, it stores the name of each city (and is unique), so we can use it as an *index* for each observation

In [ ]:
cities = cities.set_index('city')

In [ ]:
cities.head()

In [ ]:
cities_with_data = area_interpolate(source_df=oc_tracts, target_df=cities, extensive_variables=extensive_variables, intensive_variables=intensive_variables)

In [ ]:
cities_with_data

In [ ]:
cities_with_data.explore('n_total_pop', tiles='cartodb positron')

In [ ]:
cities_with_data.explore('p_edu_college_greater', scheme='quantiles', cmap='Greens', tiles='cartodb positron')

## Dasymetric Interpolation

for this example, it's useful to have a finer-resolution `target_df` than cities. Imagine we wantoed to estimate our tract data at a regular hexgrid geometry. The tobler package makes it easy to generate a hexgrid like this. All we need to do is provide the geodataframe we want to cover with hexes, and a resolution that defines the size of each hexagon

In [ ]:
oc_hexes = h3fy(oc_tracts, resolution=7)

In [ ]:
oc_hexes.explore()

with dasymetric interpolation, we recognize that the data in `source_df` polygons are probably not distributed uniformly thoughout, and we use another piece of data to try and improve the assumptions. One commonly used piece of information is data from the [National Land Cover Database (NLCD)](https://www.usgs.gov/centers/eros/science/national-land-cover-database)

The NLCD uses arial imagery and remotely sensed data to estimate land use and land cover in the U.S. at a 30m grid resolution. So one way we could improve our estimates is to restrict the source data to certain locations. In other words, we might assume that the data in `source_df` polygons is uniformly distributed *within the urbanized areas* of these polygons (since all of our data pertain to people and where they live)

![Example of NLCD Data](https://d9-wret.s3.us-west-2.amazonaws.com/assets/palladium/production/s3fs-public/styles/full_width/public/media/images/Annual_NLCD_Lnd_updated.jpg?itok=Qk47i_dP)

The NLCD data is a raster dataset, which means it is stored as pixels that take on a certain range of values. Specifically, a set of land-uses with these codes:

![](https://www.mrlc.gov/sites/default/files/NLCD_Colour_Classification_Update.jpg)

We can see that pixel values 21-24 refer to "developed areas" and we can safely ignore developed open space (like golf courses and landscaping), thus we want to limit our interpolation to areas of `source_df` covered by those values. 

for our own illustration, it might be useful to know where those places are. There's a utility function in `tobler` for this (though we dont need to use it ourselves for interpolation, it can be a helpful way of understanding how NLCD enters the picture

In [ ]:
raster_path = "s3://spatial-ucr/nlcd/landcover/nlcd_landcover_2021.tif"

In [ ]:
developed_area = extract_raster_features(cities,raster_path=raster_path, pixel_values=[22,23,24], collapse_values=True)

In [ ]:
developed_area.explore()

In [ ]:
developed_area.explore(tiles='USGS US Imagery')

first, lets do the simple area-weighted approach to transfer data into our hexes

In [ ]:
hex_data_areal = area_interpolate(source_df=oc_tracts, target_df=oc_hexes, extensive_variables=extensive_variables, intensive_variables=intensive_variables)

In [ ]:
hex_data_areal.explore('n_total_pop', tiles='cartodb positron')

Now we want to carry out the interpolation using the `masked_area_interpolation` function, which allows us to bring in the NLCD data

In [ ]:
masked_area_interpolate?

this function works the same as `area_weighted`, except we now also include the path to our raster data, and the values of the pixels that define the land-uses we care about

In [ ]:
hexes_data_dasy = masked_area_interpolate(source_df=oc_tracts, target_df=oc_hexes, extensive_variables=extensive_variables, intensive_variables=intensive_variables, raster=raster_path, pixel_values=[22,23,24])

In [ ]:
hexes_data_dasy.explore('n_total_pop', tiles='cartodb positron')

The map looks similar, but the values are actually different. To see, we can take the difference between the two approaches and compare which places had more or fewer people when using the dasymetric (masked area) approach

In [ ]:
difference = hexes_data_dasy['n_total_pop'] - hex_data_areal['n_total_pop']

In [ ]:
difference.hist()

In [ ]:
hexes_data_dasy['differences'] = difference

In [ ]:
hexes_data_dasy

In [ ]:
hexes_data_dasy.explore('differences', cmap='RdBu_r', scheme='quantiles')

In [ ]:
hexes_data_dasy.explore('differences', cmap='RdBu_r', scheme='quantiles', tiles='usgs us imagery')